# This notebook is intended to show how to use pandas, and sql alchemy to upload data into DB2-switch. 

Install using pip or any other package manager pandas, sqlalchemy and pg8000. The later one is the driver to connect to the db. 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

After importing the required packages, first create the engine to connect to the DB. The approach I generally use is to create a string based on the username and password. The code is a function, you just need to fill in with the username, password and the dbname. 

It allows you to create different engines to connect to serveral dbs. 

In [ ]:
def connection(user,passwd,dbname):
    str1 = ('postgresql+pg8000://' + user +':' + passw + '@switch-db2.erg.berkeley.edu:5432/' 
            + dbname + '?ssl=true&sslfactory=org.postgresql.ssl.NonValidatingFactory')
    engine = create_engine(str1)
    return engine

In [ ]:
user = ''
passw = ''
dbname = 'switch_cr' 
engine_switch_cr = connection(user,passw,dbname)

Afterwards, use pandas to import the data from CSV's or any other text file format. Make sure that the data in good shape before trying to push it into the server. 

In [ ]:
csvFilename = 'Trial_data.csv'
pd_data = pd.read_csv(csvFilename, encoding='UTF-8', delimiter = ',')

Now we have a connection named ```engine_source``` and a data frame with data ```pd_data```, this can be uploaded to the DB 

In [ ]:
table_name = 'test_data'
schema_for_upload = 'sandbox'
pd_data.to_sql(table_name, engine_switch_cr, schema=schema_for_upload)

You can also use sqlalchemy to get data from other DBs and then upload them to our SWITCH schema, for example we can get the table ```energy_sources``` and load it into a pandas dataframe. 

In [ ]:
#in this case I am connecting to a different db. 
engine_switch_chile = connection(user,passw,'switch_chile')
schema = 'chile_new'
table = 'energy_sources'
#The query can be modified to get a particular piece of the data, in this case is a simple * 
# to get everything. 
string_query = 'select * from ' + schema + '.' + table
pd_data = pd.read_sql_query(string_query,engine_switch_chile)
pd_data

Once the data is in the dataframe it can be modified, you can annex data or merge with other tables. As soon as the dataframe is ready, it can be pushed back into the relevant database.

In [ ]:
table_name = 'test_data2'
schema_for_upload = 'sandbox'
pd_data.to_sql(table_name, engine_switch_cr, schema=schema_for_upload)